In [1]:
import pandas as pd 
import numpy as np 

In [2]:
pd_data = pd.read_csv(
    './vegeterianl_dataset.csv',  delimiter=r"\s+")
pd_data.drop("ID",axis=1)
pd_data

,ID,STREAM,SLOPE,ELEVATION,VEGETATION
0,1,False,steep,3900,chapparal
1,2,True,moderate,300,riparian
2,3,True,steep,1500,riparian
3,4,False,steep,1200,chapparal
4,5,False,flat,4450,conifer
5,6,True,steep,5000,conifer
6,7,True,steep,3000,chapparal


In [12]:
pd_data.values

array([[1, False, 'steep', 3900, 'chapparal'],
       [2, True, 'moderate', 300, 'riparian'],
       [3, True, 'steep', 1500, 'riparian'],
       [4, False, 'steep', 1200, 'chapparal'],
       [5, False, 'flat', 4450, 'conifer'],
       [6, True, 'steep', 5000, 'conifer'],
       [7, True, 'steep', 3000, 'chapparal']], dtype=object)

In [109]:
a = np.array([['absent',6,3,5],
     ['absent',9,3,14],
              ['present',8,4,5],
              ['absent',2,5,1],
              ['absent',1,4,15],
              ['absent',1,2,16],
              ['absent',5,2,17],
              ['absent',3,3,16],
              ['absent',7,2,16],
              ['present',4,6,12]])

In [110]:
pd_data = pd.DataFrame(a)

In [111]:
pd_data.columns=["a","b","c","d"]

In [112]:
pd_data

,a,b,c,d
0,absent,6,3,5
1,absent,9,3,14
2,present,8,4,5
3,absent,2,5,1
4,absent,1,4,15
5,absent,1,2,16
6,absent,5,2,17
7,absent,3,3,16
8,absent,7,2,16
9,present,4,6,12


In [113]:
pd_data.sort_values("b")

,a,b,c,d
4,absent,1,4,15
5,absent,1,2,16
3,absent,2,5,1
7,absent,3,3,16
9,present,4,6,12
6,absent,5,2,17
0,absent,6,3,5
8,absent,7,2,16
2,present,8,4,5
1,absent,9,3,14


In [114]:
def get_info(df):
    riparian = df.loc[df["a"]=="absent"]
    chapparal = df.loc[df["a"]=="present"]

    x = np.array([len(riparian)/len(df),len(chapparal)/len(df)])
    y = np.log2(x[x!=0])
    

    info_all = - sum(x[x!=0] * y)
    return info_all

In [115]:
get_info(pd_data)
# info(D)

0.7219280948873623

In [116]:
def get_attribute_info(df, attribute_name, continuous_value=0):
    get_infos = []
    if continuous_value == 0:
        attribute_values = pd_data[attribute_name].unique()
        for value in attribute_values:
            split_df = pd_data.loc[pd_data[attribute_name] == value]
            get_infos.append((len(split_df) / len(df)) * get_info(split_df))
    else:
        split_df_1 = pd_data.loc[pd_data[attribute_name] >= continuous_value]
        split_df_2 = pd_data.loc[pd_data[attribute_name] < continuous_value]

        get_infos.append((len(split_df_1) / len(df)) * get_info(split_df_1))                     
        get_infos.append((len(split_df_2) / len(df)) * get_info(split_df_2))                     
    return sum(get_infos)

In [117]:
def get_continuos_attribute_info(df, attribute_name):
    result = pd_data.sort_values(attribute_name)
    prior = 0 
    split_index = []
    sorted_key = {}
    sorted_index = {}
    for index, value in enumerate(result.index):
        if prior != result["a"][value]:
            split_index.append(value)
            prior = result["a"][value]
        sorted_index[value] = index
        sorted_key[index] = value
    split_value = []
    for v in split_index[1:]:
        current_value = result[attribute_name][v]
        prior_value_index = sorted_key[sorted_index[v] - 1]
        prior_value = result[attribute_name][prior_value_index]
        
        split_value_average = current_value + prior_value
        split_value.append(str(int(split_value_average)/2))
    result = {}
    for value in split_value:
        result[value] = (get_attribute_info(df, attribute_name, value))
    return result

In [118]:
get_continuos_attribute_info(pd_data, "b")
# Info(elec)

{'21.5': 0.6041843979966417,
 '27.0': 0.6041843979966417,
 '43.5': 0.7219280948873623,
 '49.0': 0.7219280948873623}

In [119]:
for key, value in get_continuos_attribute_info(pd_data, "b").items():
    print(key, ":", get_info(pd_data) - value)
# Gain(elec)

21.5 : 0.11774369689072062
27.0 : 0.11774369689072062
43.5 : 0.0
49.0 : 0.0


In [120]:
get_info(pd_data) - get_attribute_info(pd_data, "c")

0.5219280948873624

In [121]:
get_info(pd_data) - get_attribute_info(pd_data, "d")

0.5219280948873624